# Estimating Queue Priority Wait Times

https://dl.acm.org/doi/10.1145/3427921.3450249

In [1]:
import pandas
from scipy.optimize import newton

In [2]:
P_GROUP_BOUNDS = [1,3,10]

COLS_TO_USE = [4,5,6,9,10,12,13,14,15,16]
HEADER_NAMES = ['size','weight','recieved_time','fee','confirmed_block_height','confirm_time','waiting_time','fee_rate','enter_block_height','no_block_confirm']

### Data Loading

In [3]:
d1 = pandas.read_csv('TimetxinBlock621500.csv', usecols=COLS_TO_USE, names=HEADER_NAMES)
d2 = pandas.read_csv('TimetxinBlock622000.csv', usecols=COLS_TO_USE, names=HEADER_NAMES)
d3 = pandas.read_csv('TimetxinBlock622500.csv', usecols=COLS_TO_USE, names=HEADER_NAMES)
dataFrame = pandas.concat([d1, d2, d3])
dataFrame = dataFrame.sort_values(by=['recieved_time'])
dataFrame.head(10)

,size,weight,recieved_time,fee,confirmed_block_height,confirm_time,waiting_time,fee_rate,enter_block_height,no_block_confirm
68059,1220,2258,1583756823,567,621031,1583810262,53439,1.004429,620943,89
67841,284,806,1583756865,202,621031,1583810262,53397,1.002481,620943,89
68124,926,1736,1583756869,436,621031,1583810262,53393,1.004608,620943,89
16126,420,1032,1583756885,258,621006,1583796952,40067,1.000000,620943,64
10295,420,1032,1583756899,258,621004,1583796307,39408,1.000000,620943,62
67965,247,661,1583756901,166,621031,1583810262,53361,1.004539,620943,89
68119,667,2668,1583756912,670,621031,1583810262,53350,1.004498,620943,89
67559,283,805,1583756927,202,621031,1583810262,53335,1.003727,620943,89
67654,247,661,1583756972,166,621031,1583810262,53290,1.004539,620943,89
67945,283,805,1583757002,202,621031,1583810262,53260,1.003727,620943,89


### Block Size

In [4]:
block_groups = dataFrame.groupby(['confirmed_block_height'])['confirmed_block_height'].count()
mean_block_size = float(round(block_groups.mean()))
print("Mean Block Size: ", mean_block_size)

Mean Block Size:  2266.0


### Fee Rate Priority Range

In [5]:
def calculate_feerate_for_priority_groups(data, upper_boundary):
    feerate_range = []
    transaction_counts = 0
    for i in range(len(P_GROUP_BOUNDS)):
        transaction_counts = len(data[data['no_block_confirm'] <= upper_boundary[i]])
        boundary = data['fee_rate'].nlargest(transaction_counts).iloc[-1]
        feerate_range.append(boundary)
    return feerate_range
            
p_groups_feerate_range = calculate_feerate_for_priority_groups(dataFrame, P_GROUP_BOUNDS)
print("one block waited ration: ", p_groups_feerate_range)

one block waited ration:  [30.17718371153248, 18.529411764705888, 5.0]


### Arrival Rates (Lambda) - Priority

In [7]:
p_groups_lambda = []
transaction_count = len(dataFrame)

for i in range(len(p_groups_feerate_range)+1):
    p_transaction_count = 0.0
    p_lambda = 0.0
    p_feerate_min = 0.0
    p_feerate_max = 0.0
    if(i == 0):
        p_df = dataFrame[dataFrame['fee_rate'] >= p_groups_feerate_range[0]]
        p_transaction_count = len(p_df)
        p_feerate_min = float(p_df['fee_rate'].min())
        p_feerate_max = float(p_df['fee_rate'].max())
        p_lambda = float(p_transaction_count) / (float(p_df['recieved_time'].max()) - float(p_df['recieved_time'].min()))
    elif(i == (len(p_groups_feerate_range))):
        p_df = dataFrame[dataFrame['fee_rate'] < p_groups_feerate_range[i-1]]
        p_transaction_count = len(p_df)
        p_feerate_min = float(p_df['fee_rate'].min())
        p_feerate_max = float(p_df['fee_rate'].max())
        p_lambda = float(p_transaction_count) / (float(p_df['recieved_time'].max()) - float(p_df['recieved_time'].min()))
    else:
        p_df = dataFrame[(dataFrame['fee_rate'] <= p_groups_feerate_range[i-1]) & (dataFrame['fee_rate'] >= p_groups_feerate_range[i])]
        p_transaction_count = len(p_df)
        p_feerate_min = float(p_df['fee_rate'].min())
        p_feerate_max = float(p_df['fee_rate'].max())
        p_lambda = float(p_transaction_count) / (float(p_df['recieved_time'].max()) - float(p_df['recieved_time'].min()))

    p_groups_lambda.append(p_lambda)

    print("Priority Group " + str(i+1))
    print("Transaction Count: ", p_transaction_count)
    print("Feerate Min: ", p_feerate_min)
    print("Feerate Max: ", p_feerate_max)
    print("Lambda: " + str(p_lambda) + "\n")

print("Sum Of Lambda: ", sum(p_groups_lambda))

Priority Group 1
Transaction Count:  2035992
Feerate Min:  30.17718371153248
Feerate Max:  52380.95238095238
Lambda: 1.8686986243533874

Priority Group 2
Transaction Count:  565762
Feerate Min:  18.529411764705888
Feerate Max:  30.17718371153248
Lambda: 0.5165653492665521

Priority Group 3
Transaction Count:  389945
Feerate Min:  5.0
Feerate Max:  18.529411764705888
Lambda: 0.3554804585797738

Priority Group 4
Transaction Count:  397918
Feerate Min:  0.907563025210084
Feerate Max:  4.998487140695915
Lambda: 0.3619598250955343

Sum Of Lambda:  3.1027042572952475


### Service Rate (Mu)

In [8]:
service_time = 600
#mu = 1/service_time
mu = (1/service_time)*mean_block_size
#mu = mean_block_size / service_time
print("Mu: ", mu)

Mu:  3.776666666666667


### Validation Of Z Using Newton Method (Mine Vs SciPy)

In [9]:
my_p_group_z = []
sciPy_p_group_z = []
lambda_sum = 0.0

def NewtonMethod(lam, m_u, block_size, x0, epsilon):
    fx = lambda x: lam*(1 - x) - m_u*x*(1 - x**block_size)
    dfx = lambda x: m_u*(block_size*x**block_size + x**block_size - 1) - lam
    while True:
        x1 = x0 - fx(x0) / dfx(x0)
        t = abs(x1 - x0)
        if t < epsilon:
            break
        x0 = x1
    return x0

def SciPyNewton(lam, m_u, block_size, x0, epsilon, max_iteration):
    fx = lambda x: lam*(1 - x) - m_u*x*(1 - x**block_size)
    #dfx = lambda x: m_u*(block_size*x**block_size + x**block_size - 1) - lam
    return newton(fx, x0, fprime=None, args=(), tol=epsilon, maxiter=max_iteration, fprime2=None)

for i in range(len(p_groups_lambda)):
    lambda_sum += p_groups_lambda[i]
    my_p_group_z.append(NewtonMethod(lambda_sum, mu, mean_block_size, 0, 1e-10))
    sciPy_p_group_z.append(SciPyNewton(lambda_sum, mu, mean_block_size, 0, 1e-10, 500))

for i in range(len(my_p_group_z)):
    print("Priority Group " + str(i+1))
    print("My Z: \n" + str(my_p_group_z[i]))
    print("SciPy Z: \n" + str(sciPy_p_group_z[i]) + "\n")
    

Priority Group 1
My Z: 
0.3310146514922393
SciPy Z: 
0.3310146514922393

Priority Group 2
My Z: 
0.38709685533055516
SciPy Z: 
0.3870968553305551

Priority Group 3
My Z: 
0.42052655427496827
SciPy Z: 
0.42052655427496827

Priority Group 4
My Z: 
0.4510156948345448
SciPy Z: 
0.45101569483454484



### Wait Times Using Z

In [10]:
prev_transaction_in_queue = 0.0
p_group_response_time = []

for i in range(len(p_groups_lambda)):
    transaction_in_queue = my_p_group_z[i] / (1 - my_p_group_z[i]) - prev_transaction_in_queue # L = z / (1 - z) - (Previous L)
    response_time  = transaction_in_queue / p_groups_lambda[i] # W = L / Lambda

    prev_transaction_in_queue = transaction_in_queue

    p_group_response_time.append(response_time)

    print("Priority Group " + str(i+1))
    print("Z: ", my_p_group_z[i])
    print("Average Transaction In Queue (L): ", transaction_in_queue)
    print("Average Response Time In Queue (W): " + str(response_time) + "\n")

Priority Group 1
Z:  0.3310146514922393
Average Transaction In Queue (L):  0.49480104793117047
Average Response Time In Queue (W): 0.26478375992939096

Priority Group 2
Z:  0.38709685533055516
Average Transaction In Queue (L):  0.1367781154280368
Average Response Time In Queue (W): 0.2647837599293911

Priority Group 3
Z:  0.42052655427496827
Average Transaction In Queue (L):  0.5889265003353472
Average Response Time In Queue (W): 1.6567056954079673

Priority Group 4
Z:  0.4510156948345448
Average Transaction In Queue (L):  0.23261919886021676
Average Response Time In Queue (W): 0.6426657952959838

